Hello, I am JYC. This notebook implements all the sub-tasks in experimental topic 3. I will explain the logic behind each concept to assist students who are not proficient in coding to quickly grasp the key points and complete the experiment.

This part is rely on a multi-agent simulation library **agentsim** which is developed by myself. You should install agentsim first following [here](https://github.com/Kami-chanw/agentsim).

In this experiment, I implemented both a centralized coordination method and a multi-agent reinforcement learning (MARL) method. The first scenario is based on maze cleaning, while the second scenario is based on the Prisoner's Dilemma game.

In [1]:
from agentsim.maze import MazeWindow, MazeCleanEnv
from agentsim.pdgame import PDGameEnv, PDGameWindow

import pyglet

### Centralized Coordination Method

In the maze cleaning environment, agents are tasked with exploring and cleaning a maze as efficiently as possible. The maze is represented as an undirected, unweighted, acyclic graph, where nodes represent locations and edges represent paths between locations. It should be implemented with IPPO, but I don't have much time to learn it...

The primary objectives of this environment are:
- To ensure that each agent explores and cleans as many new locations as possible.
- To minimize the overlap of cleaned locations between agents to avoid redundant cleaning.
- To optimize the overall cleaning time by efficiently coordinating the agents' movements to cover the entire maze.

Agents receive rewards based on their actions:
- A reward for exploring a new location.
- A penalty for revisiting an already cleaned location.
- A significant reward for completing the cleaning of the entire maze.

The centralized coordination method involves a central controller that directs the actions of all agents, which is totally based on search algorithm. The controller has a global view of the maze and the status of all agents, allowing it to make informed decisions to achieve the objectives efficiently. 

Its algorithmic process is as follows:

1. Perform spectral decomposition on the graph composed of feasible paths in the maze, resulting in `n_agents` subgraphs.
2. For each subgraph, arbitrarily select a leaf node `u`, then use breadth-first search (BFS) to find the node `v` that is the furthest from `u`, and set `v` as the starting node.
3. Start from `v` to traverse the entire graph, preprocessing the height of each node. The height here is defined as the maximum height of all child nodes plus one. Specifically, the height of a leaf node is 0.
4. Continue traversing the entire graph from `v`. When a node `w` has a degree greater than 2 (indicating multiple branches), prioritize the branch with the smaller height.

In [2]:
env = MazeCleanEnv(5, 5, 5)
window = MazeWindow(
    env,
    width=1000,
    height=800,
    caption="Maze Cleaner Simulation",
    solve_interval=0.5,
)

pyglet.app.run()

### Multi-Agent Reinforcement Learning Method

#### Prisoner's Dilemma Game Environment

In the Prisoner's Dilemma (PD) game environment, multiple agents interact with each other in a series of two-player games. Each game round involves two agents making a choice: to cooperate (pay a coin) or to defect (not pay a coin). The outcome of their choices is determined by a reward matrix:

- If both players cooperate, they both receive a reward.
- If one cooperates and the other defects, the defector receives a higher reward, while the cooperator receives nothing.
- If both defect, neither receives a reward.

The PD game environment is characterized by the dilemma that while mutual cooperation yields the highest collective reward, individual rationality often leads to defection, resulting in suboptimal outcomes for both players.

#### Reinforcement Learning Player: Q-Learner

In this environment, a simple reinforcement learning player called the Q-Learner is employed. The Q-Learner's strategy is based on Q-Learning, a model-free reinforcement learning algorithm. Q-Learning works as follows:

- **State Representation**: The state can be represented by the current strategy of the opponent (e.g., frequency of cooperation and defection).
- **Action Space**: The possible actions are to cooperate or defect.
- **Reward Function**: The reward received depends on the outcomes of the PD game rounds.
- **Q-Table**: The Q-Learner maintains a Q-table where each state-action pair has an associated Q-value, representing the expected future rewards of taking that action in that state.
- **Update Rule**: After each game round, the Q-Learner updates its Q-values using the formula:
  $$
  Q(s, a) \leftarrow Q(s, a) + \alpha \left[ r + \gamma \max_{a'} Q(s', a') - Q(s, a) \right]
  $$
  where $s$ is the current state, $a$ is the action taken, $r$ is the reward received, $s'$ is the next state, $\alpha$ is the learning rate, and $\gamma$ is the discount factor.
- **Policy**: The Q-Learner follows an $\varepsilon$-greedy policy, where with probability $\varepsilon$ it chooses a random action (exploration), and with probability $1-\varepsilon$ it chooses the action with the highest Q-value (exploitation).

By using the Q-Learner in the PD game environment, we can explore how individual agents learn to cooperate or defect based on their experiences and how their strategies evolve over time through repeated interactions. This setup allows for the analysis of emergent behaviors and the effectiveness of reinforcement learning in multi-agent settings.

In [2]:
reward_matrix = [[(2, 2), (-1, 3)], [(3, -1), (0, 0)]]

role_num_dict = {"copycat": 5, "cooperator": 5, "fraud": 5, "grudger": 5, "qlearner": 20}
env = PDGameEnv(reward_matrix, role_num_dict, 5)
window = PDGameWindow(env)
pyglet.app.run()